In [1]:
import pandas as pd
from des_helpers import select_function, dpa_select_function, encrypt, decrypt, bin2hex, des, preprocess_key, dec2bin

# Load data
trace_df = pd.read_csv("des_extension_data.csv")

ciphertexts = [c.upper() for c in trace_df['Ciphertext']]
plaintexts = [p.upper() for p in trace_df['Plaintext']]

In [2]:
trace_df.iloc[0]

Key                          0
Plaintext     0000000000000000
Ciphertext    8CA64DE9C1B123A7
Power                     -376
Name: 0, dtype: object

In [3]:
# Test encrypt decrypt
sample = trace_df.iloc[0]
key = f"{0:016x}".upper()
pt = sample["Plaintext"].upper()
ct = sample["Ciphertext"].upper()

text = bin2hex(encrypt(pt, key))
print(text == ct)

text = bin2hex(decrypt(ct, key))
print(text == pt)

True
True


In [ ]:
# Test select func
def mod_decrypt(ciphertext, key):
	rkb = preprocess_key(key)
	rkb_rev = rkb[::-1]
	text, sbox_out = des(ciphertext, rkb_rev)

	return text, sbox_out

def mod_encrypt(plaintext, key):
	rkb = preprocess_key(key)
	text, sbox_out = des(plaintext, rkb)

	return text, sbox_out

for i in range(10):
	ct = ciphertexts[i]
	pt = plaintexts[i]

	text, sbox_out = mod_decrypt(ct, key)
	print(f"Decryption S-Box Output for last round first s-box: {dec2bin(sbox_out[15][0])}")

	text, sbox_out = mod_encrypt(pt, key)
	print(f"Encryption S-Box Output for first round first s-box: {dec2bin(sbox_out[0][0])}")

Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output for first round first s-box: 1110
Decryption S-Box Output for last round first s-box: 1110
Encryption S-Box Output

In [12]:
import numpy as np


for key in range(0,64):
	out = []

	for pt in plaintexts:
		sbox_out = select_function(pt, key)
		out.append({"pt": pt, "s-box out": sbox_out})

	print(f"S-box outs for key = {key}")
	print(pd.DataFrame(out)["s-box out"].value_counts())

S-box outs for key = 0
s-box out
1110    512
0011    512
Name: count, dtype: int64
S-box outs for key = 1
s-box out
0000    512
1010    512
Name: count, dtype: int64
S-box outs for key = 2
s-box out
0100    512
1010    512
Name: count, dtype: int64
S-box outs for key = 3
s-box out
1111    512
0110    512
Name: count, dtype: int64
S-box outs for key = 4
s-box out
1101    512
0110    512
Name: count, dtype: int64
S-box outs for key = 5
s-box out
0111    512
1100    512
Name: count, dtype: int64
S-box outs for key = 6
s-box out
0001    512
1100    512
Name: count, dtype: int64
S-box outs for key = 7
s-box out
0100    512
1011    512
Name: count, dtype: int64
S-box outs for key = 8
s-box out
0010    512
0101    512
Name: count, dtype: int64
S-box outs for key = 9
s-box out
1110    512
1001    512
Name: count, dtype: int64
S-box outs for key = 10
s-box out
1111    512
1001    512
Name: count, dtype: int64
S-box outs for key = 11
s-box out
0010    512
0101    512
Name: count, dtype: int64
S-

In [6]:
# Test on entire dataset
results = []
for pt in plaintexts:
	text, sbox_out = mod_encrypt(pt, key)
	encrypt_out = int(dec2bin(sbox_out[0][0])[0])
	select_out = dpa_select_function(pt, 0) # correct key is 60 from the paper
	
	if encrypt_out == select_out: 
		results.append(True)
	else:
		print(encrypt_out, select_out)
		results.append(False)
		
print(all(results))

True
